In [28]:
import pandas as pd
import re
import emoji
import string
from emoticon_lib import replace_emoticons_with_text
from deep_translator import (GoogleTranslator)
import html
from nltk import word_tokenize
from nltk.corpus import stopwords
import contractions
#import sys
#print(sys.executable)
#%pip install wordcloud
#%pip install contractions

In [29]:
df = pd.read_csv('cyberbullying_tweets.csv')

DATA UNDERSTANDING

In [5]:
df.head()

,tweet_text,cyberbullying_type
0,"In other words #katandandre, your food was cra...",not_cyberbullying
1,Why is #aussietv so white? #MKR #theblock #ImA...,not_cyberbullying
2,@XochitlSuckkks a classy whore? Or more red ve...,not_cyberbullying
3,"@Jason_Gio meh. :P thanks for the heads up, b...",not_cyberbullying
4,@RudhoeEnglish This is an ISIS account pretend...,not_cyberbullying


In [19]:
df.tail()

,tweet_text,cyberbullying_type
47687,"Black ppl aren't expected to do anything, depe...",ethnicity
47688,Turner did not withhold his disappointment. Tu...,ethnicity
47689,I swear to God. This dumb nigger bitch. I have...,ethnicity
47690,Yea fuck you RT @therealexel: IF YOURE A NIGGE...,ethnicity
47691,Bro. U gotta chill RT @CHILLShrammy: Dog FUCK ...,ethnicity


In [13]:
df.shape

(47692, 2)

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 47692 entries, 0 to 47691
Data columns (total 2 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   tweet_text          47692 non-null  object
 1   cyberbullying_type  47692 non-null  object
dtypes: object(2)
memory usage: 745.3+ KB


In [7]:
df.describe()

,tweet_text,cyberbullying_type
count,47692,47692
unique,46017,6
top,RT @sailorhg: the intro for my hardware hackin...,religion
freq,2,7998


In [8]:
df.isna().sum()

tweet_text            0
cyberbullying_type    0
dtype: int64

In [10]:
# Unique values of target variable
df.cyberbullying_type.unique()

array(['not_cyberbullying', 'gender', 'religion', 'other_cyberbullying',
       'age', 'ethnicity'], dtype=object)

In [12]:
# Count of values of target variable
df['cyberbullying_type'].value_counts()

religion               7998
age                    7992
gender                 7973
ethnicity              7961
not_cyberbullying      7945
other_cyberbullying    7823
Name: cyberbullying_type, dtype: int64

In [18]:
print('Num of Duplicated Rows:', df.duplicated().sum())

Num of Duplicated Rows: 0


DATA CLEANING

In [17]:
# Remove Duplicates
df =df.drop_duplicates(subset='tweet_text', keep="last")

In [33]:
# Expand contraction
df['tweet_text'] = df['tweet_text'].apply(lambda e: contractions.fix(e))
# df.head(10)
print(df.head(10).to_markdown())

|    | tweet_text                                                                                                                          | cyberbullying_type   |
|---:|:------------------------------------------------------------------------------------------------------------------------------------|:---------------------|
|  0 | In other words #katandandre, your food was crapilicious! #mkr                                                                       | not_cyberbullying    |
|  1 | Why is #aussietv so white? #MKR #theblock #ImACelebrityAU #today #sunrise #studio10 #Neighbours #WonderlandTen #etc                 | not_cyberbullying    |
|  2 | @XochitlSuckkks a classy whore? Or more red velvet cupcakes?                                                                        | not_cyberbullying    |
|  3 | @Jason_Gio meh. :P  thanks for the heads up, but not too concerned about another angry dude on twitter.                             | not_cyberbullying    |
|  4 | @RudhoeEn

In [ ]:
# Handle abbreviation    #jy

In [ ]:
# Word Segmentation    # DONT DO THIS!!!!
import wordninja
df['tweet_text'] = df['tweet_text'].apply(lambda x: ' '.join(wordninja.split(x)))
df.head(10)    

In [ ]:
#Remove html tags
df['tweet_text'] = df['tweet_text'].apply(lambda e: html.unescape(e))

In [ ]:
## Remove RT, #, URL and @:   #DOUBLE CHECK
df['tweet_text'] = df['tweet_text'].apply(lambda e: re.sub(r"(RT)|(:)|(@)|(?:\@|\#|https?|http?|www)\S+", "", e))

In [ ]:
## double check if removing special character can handle # and @ tgt. If yes, no need to use regex

In [ ]:
# Remove email 
df['tweet_text'] = df['tweet_text'].apply(lambda e: re.sub(r"\S+@\S+(?:\.\S+)+", "", e))

In [ ]:
# Change emojis to text
df['tweet_text'] = df['tweet_text'].apply(lambda e: emoji.demojize(e, language="en"))

In [ ]:
# Remove emoticons
df['tweet_text'] = df['tweet_text'].apply(lambda e: replace_emoticons_with_text(e))

In [ ]:
# Remove digits
df['tweet_text'] = df['tweet_text'].replace(to_replace=r'\d', value='', regex=True)

In [ ]:
# Translate
df['tweet_text'] = df['tweet_text'].apply(lambda e: GoogleTranslator(source='auto', target='en').translate(text = e))

In [ ]:
# Check for rows with nan or empty spaces   --chloe   

In [ ]:
# Remove special characters    # str.replace('username',"")
#replace, regex

In [ ]:
# Remove punctuations
def remove_punctuations(text):
    for char in string.punctuation:
        text = text.replace(char, '')
    return text
df['tweet_text'] = df['tweet_text'].apply(remove_punctuations)
print(df.head(10).to_markdown())

In [ ]:
# Remove extra whitespaces after removing noise and punctuations
def remove_extra_whitespaces(text):
    # Replace multiple consecutive whitespaces with a single space
    text = re.sub(r'\s+', ' ', text) 
    return text.strip()  # Remove leading and trailing whitespaces
df['tweet_text'] = df['tweet_text'].apply(remove_extra_whitespaces)

In [ ]:
# Remove repeating characters (use regex)  **the num of characters in a word shdn't >3 #mommmmmmmmmmmmmm

In [ ]:
# Convert to lower case
df['tweet_text'] = df['tweet_text'].str.lower()

In [ ]:
# Tokenization
df['tweet_text'] = df.apply(lambda row: word_tokenize(row['tweet_text']), axis=1)
print(df.head(10).to_markdown())

# Remove stopwords
stopword = stopwords.words('english')
df['tweet_text'] = df['tweet_text'].apply(lambda e: ' '.join([word for word in e.split() if word not in (stopword)]))
print(df.head(10).to_markdown())

In [ ]:
# Lemmatization
lemmatizer = WordNetLemmatizer()
def lemmatize_words(text):
    return ' '.join([lemmatizer.lemmatize(word) for word in text.split()])
df["tweet_text"] = df["tweet_text"].apply(lambda text: lemmatize_words(text))

In [ ]:
# Save processed dataset into new csv file
df.to_csv('new_cyberbully_tweet.csv', index=False)